In [1]:
!pip install --upgrade pip

  Using cached pip-24.3.1-py3-none-any.whl.metadata (3.7 kB)
Using cached pip-24.3.1-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2


In [2]:
!pip install langchain

In [3]:
!pip install langchain-community langchain-google-genai sentence-transformers faiss-cpu

  Using cached langchain_google_genai-2.0.4-py3-none-any.whl.metadata (3.8 kB)
  Using cached google_generativeai-0.8.3-py3-none-any.whl.metadata (3.9 kB)
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_google_genai-2.0.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached langchain_google_genai-2.0.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.7.2-py3-none-any.whl.metadata (4.0 kB)
  Using cached langchain_google_genai-1.0.10-py3-none-any.whl.metadata (3.8 kB)
  Using cached google_ai_generativelanguage-0.6.6-py3-none-any.whl.metadata (5.6 kB)
  Using cached google_api_python_client-2.151.0-py2.py3-none-any.whl.metadata (6.7 kB)
  Using cached google_auth-2.36.0-py2.py3-none-any.whl.metadata (4.7 kB)
  Using cached proto_plus-1.25.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached langsmith-0.1.142-py3-none-any.whl.metadata 

In [4]:
# 필요한 라이브러리 임포트
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
# from langchain.chat_models import ChatOpenAI
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
# from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

# FAISS와 임베딩 모델 초기화
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# 환경 변수 로드
load_dotenv()

# gemini-1.5-flash 모델 설정
gemini_1_5_flash = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.0)
gemini_pro = ChatGoogleGenerativeAI(model="gemini-pro")

# 로컬 데이터베이스 로드

embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
db = FAISS.load_local('./db/on-site-first-aid', embeddings_model, allow_dangerous_deserialization=True)

# RAG 체인 생성
retriever = db.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 5, 'lambda_mult': 0.15}
)

# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

ValidationError: 1 validation error for ChatGoogleGenerativeAI
__root__
  Did not find google_api_key, please add an environment variable `GOOGLE_API_KEY` which contains it, or pass `google_api_key` as a named parameter. (type=value_error)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [3]:
llm = gemini_1_5_flash
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "벌에 쏘였을 때 응급처치 방법 알려주세요."})
response["answer"]

'벌에 쏘였을 때는 먼저 벌침을 제거해야 합니다. 핀셋을 사용하지 말고 신용카드 등 딱딱한 물체로 긁어내듯이 제거하는 것이 좋습니다. 벌침을 제거한 후에는 쏘인 부위를 깨끗한 물로 씻고 얼음찜질을 해 주세요. 붓기가 심하거나 통증이 심할 경우에는 병원을 방문하여 진료를 받으세요. \n'

In [4]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "벌에 쏘였을 때는 어떻게 해야하나요?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

In [5]:
second_question = "붓기가 지속되면 어떻게하면 되나요?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

붓기가 지속되면 냉찜질을 계속 해주는 것이 좋습니다. 냉찜질은 붓기를 가라앉히고 통증을 완화하는 데 도움이 됩니다. 냉찜질은 하루에 여러 번, 한 번에 15~20분 정도 해주는 것이 좋습니다. 냉찜질을 해도 붓기가 가라앉지 않거나, 통증이 심하거나, 다른 증상이 나타나면 병원을 방문하여 진료를 받아야 합니다. 



In [108]:
# !pip install langgraph

In [6]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


# We define a dict representing the state of the application.
# This state has the same input and output keys as `rag_chain`.
class State(TypedDict):
    input: str
    chat_history: Annotated[Sequence[BaseMessage], add_messages]
    context: str
    answer: str


# We then define a simple node that runs the `rag_chain`.
# The `return` values of the node update the graph state, so here we just
# update the chat history with the input message and response.
def call_model(state: State):
    response = rag_chain.invoke(state)
    return {
        "chat_history": [
            HumanMessage(state["input"]),
            AIMessage(response["answer"]),
        ],
        "context": response["context"],
        "answer": response["answer"],
    }


# Our graph consists only of one node:
workflow = StateGraph(state_schema=State)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Finally, we compile the graph with a checkpointer object.
# This persists the state, in this case in memory.
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

result = app.invoke(
    {"input": question},
    config=config,
)
print(result["answer"])

result = app.invoke(
    {"input": "부기가 지속된다면 어떻게 해야하나요?"},
    config=config,
)
print(result["answer"])
#벌에 쏘인 후 부기가 지속된다면, 얼음찜질을 계속 해주는 것이 도움이 됩니다. 또한, 쏘인 부위를 높게 유지하고, 필요에 따라 진통제를 복용하는 것도 도움이 될 수 있습니다. 만약 부기가 심하거나, 쏘인 부위에 통증이나 발적이 심해지면, 병원을 방문하여 진료를 받는 것이 좋습니다.

벌에 쏘였을 때는 벌침을 제거하고, 쏘인 부위를 깨끗한 물로 씻어야 합니다. 붓기와 통증을 완화하기 위해 얼음찜질을 하고, 필요에 따라 진통제를 복용할 수 있습니다. 

부기가 지속된다면, 냉찜질을 계속 해주고,  상태가 호전되지 않거나 심해지면 병원을 방문하여 진료를 받는 것이 좋습니다. 



In [7]:
chat_history = app.get_state(config).values["chat_history"]
for message in chat_history:
    message.pretty_print()

================================ Human Message =================================

벌에 쏘였을 때는 어떻게 해야하나요?
================================== Ai Message ==================================

벌에 쏘였을 때는 벌침을 제거하고, 쏘인 부위를 깨끗한 물로 씻어야 합니다. 붓기와 통증을 완화하기 위해 얼음찜질을 하고, 필요에 따라 진통제를 복용할 수 있습니다.
================================ Human Message =================================

부기가 지속된다면 어떻게 해야하나요?
================================== Ai Message ==================================

부기가 지속된다면, 냉찜질을 계속 해주고,  상태가 호전되지 않거나 심해지면 병원을 방문하여 진료를 받는 것이 좋습니다.
